In [1]:
moviedb_api_key = "8e19741f71605c762cad76ed2d1c67cb"

In [2]:
import requests
import json
moviedb_base_url = "https://api.themoviedb.org/3/movie"
moviedb_api_key = "8e19741f71605c762cad76ed2d1c67cb"
def get_movie_info(movie_id):
    movie_response = requests.get(f"{moviedb_base_url}/{movie_id}?api_key={moviedb_api_key}")
    return json.loads(movie_response.text)

In [3]:
print(get_movie_info(88))

{'adult': False, 'backdrop_path': '/6iDQCBUSZKzY8Rnu8ECwl2gXcfG.jpg', 'belongs_to_collection': {'id': 86058, 'name': 'Dirty Dancing Collection', 'poster_path': '/gkrOvdbCFuGlxVwCYmczw7aw6Ku.jpg', 'backdrop_path': '/qR6POAKwCL8VwuK8ZjSAdRRFNj9.jpg'}, 'budget': 6000000, 'genres': [{'id': 18, 'name': 'Drama'}, {'id': 10402, 'name': 'Music'}, {'id': 10749, 'name': 'Romance'}], 'homepage': 'http://lionsgateathome.com/dirty-dancing', 'id': 88, 'imdb_id': 'tt0092890', 'origin_country': ['US'], 'original_language': 'en', 'original_title': 'Dirty Dancing', 'overview': "Expecting the usual tedium that accompanies a summer in the Catskills with her family, 17-year-old Frances 'Baby' Houseman is surprised to find herself stepping into the shoes of a professional hoofer—and unexpectedly falling in love.", 'popularity': 11.925, 'poster_path': '/9Jw6jys7q9gjzVX5zm1z0gC8gY9.jpg', 'production_companies': [{'id': 137, 'logo_path': None, 'name': 'Great American Films Limited Partnership', 'origin_country

In [4]:
##Para que se vea mejor la estructura de datos
movie_id_prueba = input("id de la pelicula")
movie_response = requests.get(f"{moviedb_base_url}/{movie_id_prueba}?api_key={moviedb_api_key}")
movie_json = json.loads(movie_response.text)
for key, value in movie_json.items():
    print(key,":",value)

id de la pelicula 33


adult : False
backdrop_path : /5zkrI7P70j8C03ThBOvrEG6VF8M.jpg
belongs_to_collection : None
budget : 14400000
genres : [{'id': 37, 'name': 'Western'}]
homepage : 
id : 33
imdb_id : tt0105695
origin_country : ['US']
original_language : en
original_title : Unforgiven
overview : William Munny is a retired, once-ruthless killer turned gentle widower and hog farmer. To help support his two motherless children, he accepts one last bounty-hunter mission to find the men who brutalized a prostitute. Joined by his former partner and a cocky greenhorn, he takes on a corrupt sheriff.
popularity : 5.4195
poster_path : /54roTwbX9fltg85zjsmrooXAs12.jpg
production_companies : [{'id': 174, 'logo_path': '/ingPVoHnINIrFR7WHmYux1f5pES.png', 'name': 'Warner Bros. Pictures', 'origin_country': 'US'}, {'id': 171, 'logo_path': '/bqLWwuNCHcY3IjB5e4o29h7l80D.png', 'name': 'Malpaso Productions', 'origin_country': 'US'}]
production_countries : [{'iso_3166_1': 'US', 'name': 'United States of America'}]
release_date

In [5]:
import os
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

##Extracción
N = 150 #Número de películas a descargar
movie_ids = random.sample(range(1, 900000), N)
movies_raw =[]

for movie_id in tqdm(movie_ids):
    try:
        data = get_movie_info(movie_id)
        if data is not None:
            movies_raw.append(data)
    except Exception as e:
        print(f"Error en ID {movie_id}:{e}")
    time.sleep(0.2) #Delay para evitar muchas peticiones seguidas a TMDB

def clean_movie_data(movie):
    return {
        "id": movie.get("id"),
        "title": movie.get("title"),
        "original_title": movie.get("original_title"),
        "overview": movie.get("overview"),
        "release_date": movie.get("release_date"),
        "year": int(movie["release_date"][:4]) if movie.get("release_date") else None,
        "budget": movie.get("budget"),
        "revenue": movie.get("revenue"),
        "profit": movie.get("revenue", 0) - movie.get("budget",0),
        "runtime": movie.get("runtime"),
        "popularity": movie.get("popularity"),
        "vote_average": movie.get("vote_average"),
        "vote_count": movie.get("vote_count"),
        "genres":",".join(g["name"] for g in movie.get("genres",[])),
        "production_companies":",".join([c["name"] for c in movie.get("production_companies",[])]),
        "production_countries":",".join([c["name"] for c in movie.get("production_countries",[])]),
        "spoken_languages":",".join([l["name"] for l in movie.get("spoken_languages",[])]),
        }
clean_data = [clean_movie_data(m) for m in movies_raw]
df = pd.DataFrame(clean_data)

100%|██████████| 150/150 [01:00<00:00,  2.46it/s]


In [8]:
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")

numeric_columns = ["budget", "revenue", "profit", "runtime", "popularity", "vote_average", "vote_count"]
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors = "coerce")

df.replace(["", " ", "-", "null"], np.nan, inplace = True) #Reemplaza valores vacios

df.drop_duplicates(subset="id", inplace = True) #Elimina duplicados

df.info()
df.head()
os.makedirs("data", exist_ok = True)
df.to_csv("data/moviees_clean.csv", index = True)

print("Dataset exportado: data/movies_clean.csv")

<class 'pandas.core.frame.DataFrame'>
Index: 118 entries, 0 to 149
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    117 non-null    float64       
 1   title                 117 non-null    object        
 2   original_title        117 non-null    object        
 3   overview              99 non-null     object        
 4   release_date          116 non-null    datetime64[ns]
 5   year                  116 non-null    float64       
 6   budget                117 non-null    float64       
 7   revenue               117 non-null    float64       
 8   profit                118 non-null    int64         
 9   runtime               117 non-null    float64       
 10  popularity            117 non-null    float64       
 11  vote_average          117 non-null    float64       
 12  vote_count            117 non-null    float64       
 13  genres                90 